In [ ]:
import requests
import boto3
import json
import os
from datetime import datetime
from botocore.exceptions import NoCredentialsError

API_URL = "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
TMP_FILE_PATH = f"/tmp/datausa_population.json"  # Lambda-compatible path

def fetch_population_data(api_url, local_path):
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            with open(local_path, 'w') as f:
                json.dump(data, f, indent=4)
            print(f"Data fetched and saved to: {local_path}")
            return True
        else:
            print(f"API request failed: {response.status_code}")
            return False
    except Exception as e:
        print(f"Fetch error: {e}")
        return False

def upload_to_s3(local_path, bucket, key):
    try:
        s3 = boto3.client('s3')
        s3.upload_file(local_path, bucket, key)
        print(f"Uploaded to S3: s3://{bucket}/{key}")
        return True
    except NoCredentialsError:
        print("Credential error.")
        return False
    except Exception as e:
        print(f"Upload error: {e}")
        return False

def lambda_handler(event, context):
    bucket = event.get('bucket', 'rearc-data-quest-s3')
    s3_key = event.get(
        's3_key',
        f"population-data/datausa_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"
    )

    if fetch_population_data(API_URL, TMP_FILE_PATH):
        if upload_to_s3(TMP_FILE_PATH, bucket, s3_key):
            return {
                'statusCode': 200,
                'body': f'Successfully uploaded to s3://{bucket}/{s3_key}'
            }
        else:
            return {'statusCode': 500, 'body': 'Upload failed'}
    else:
        return {'statusCode': 500, 'body': 'Data fetch failed'}
